In [1]:
##  Importamos librerias
import numpy as np
import pandas as pd
import sys
sys.path.append("C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Lesiones")
import aux_func
import datetime
import calendar

pd.options.display.float_format = '{:.2f}'.format

In [2]:
##  Vamos a definir fecha de inicio y fecha fin del intervalo
fecha_ini = '2024-04-07'
fecha_fin = str(datetime.date.today())

Lo primero que hacemos es sacar el record de los equipos durante ese periodo

In [ ]:
##  Leemos el csv donde tengamos guardada la clasificacion en la fecha inicial
url_ini = 'C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Stats/clasi' + fecha_ini + '.csv'
clasi_ini = pd.read_csv(url_ini,sep=',')

In [ ]:
##  Scrapeamos los datos actuales
current_week = aux_func.scrapping_clasi()

##  El formato del nombre de equipo es un truño, lo mas sencillo es quedarnos con el nombre del GM, que es siempre la ultima palabra del string
current_week['GM'] = current_week['Equipo'].str.split(' ')
current_week['GM'] = current_week['GM'].apply(lambda x: x[-1])

## Seleccionamos los datos que nos interesan
clasi_final = current_week[['Equipo.1','V','GM']]
clasi_final.columns = ['V', 'D', 'GM']

##  Guardamos csv
clasi_final.to_csv('C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Stats/clasi' + str(datetime.date.today()) + '.csv')

In [ ]:
##  Si fecha final no coincide con fecha actual:
url_fin = 'C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Stats/clasi' + fecha_fin + '.csv'
clasi_final = pd.read_csv(url_fin)

In [ ]:
## Hacemos merge para sacar diferencias
total_data = clasi_final.merge(clasi_ini, on='GM',how = 'left')
total_data['V_week'] = total_data['V_x'] - total_data['V_y']
total_data['D_week'] = total_data['D_x'] - total_data['D_y']

data_final = total_data[['GM','V_week','D_week']]

In [ ]:
##  Escribimos el mes para el que estamos sacando stats
today = datetime.date.today()
first = today.replace(day=1)
last_month = first - datetime.timedelta(days=1)
mes = calendar.month_name[last_month.month]

##  Guardamos record
data_final.to_csv('C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Stats/record_' + fecha_ini + '-' + fecha_fin + '.csv')

Usamos el numero de partidos jugados para filtrar qué equipos no se tienen en cuenta

In [ ]:
data_final['n_partidos'] = data_final['V_week'] + data_final['D_week'] 
data_final.head()

In [ ]:
##   Sacamos la media de partidos jugados
media = data_final['n_partidos'].mean()

## El 75% de esa media
min_partidos = round(media * 0.75)

##  Descartamos los equipos que hayan jugado menos de ese valor
descartados = data_final[data_final['n_partidos'] < min_partidos]

In [ ]:
min_partidos

In [ ]:
descartados

Pasamos ya a sacar las stats de los jugadores

In [2]:
##  Scrapeamos stats jugadores
season = '2023-2024'
stats_players = aux_func.scrapping_stats_players(season, 27)

stats_players.to_csv('C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Stats/stats_players' + str(datetime.date.today()) + '.csv')

Scrappeando estadisticas de jugadores. Espere...


100%|██████████| 27/27 [02:35<00:00,  5.78s/it]


: 

In [4]:
##  Cargamos las stats del periodo anterior
##  Leemos el csv donde tengamos guardada la clasi de la semana pasada
url = 'C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Stats/stats_players'+ fecha_ini +'.csv'
last_stats = pd.read_csv(url)

In [5]:
##  Hacemos merge para tener ambos conjuntos en un mismo DF
merged_stats = stats_players.merge(last_stats, on='Jugador',how = 'left')

In [6]:
merged_stats

,Unnamed: 0_x,Jugador,POS_x,Equipo_x,Edad_x,PJ_x,PT_x,MIN_x,PTS_x,FGM_x,...,TL%_y,R.O_y,R.D_y,REB_y,AST_y,ROB_y,PER_y,TAP_y,FP_y,+/-_y
0,NaN,"Caín , Jamal",Alero,TOR,25,31,0,375,101,38,...,56.00,3.00,33.00,36.00,27.00,10.00,1.00,3.00,28.00,-14.00
1,NaN,"Quiñones, Lester",Escolta,TOR,23,33,6,556,274,93,...,71.40,4.00,35.00,39.00,65.00,6.00,16.00,3.00,23.00,-25.00
2,NaN,"Tshiebwe, Óscar",Ala-Pivot,DAL,24,16,1,233,57,25,...,53.80,36.00,70.00,106.00,12.00,1.00,4.00,6.00,9.00,56.00
3,NaN,"Vukcevic , Tristán",Pivot,DET,21,12,0,224,91,30,...,80.00,3.00,34.00,37.00,11.00,3.00,3.00,7.00,12.00,12.00
4,NaN,"Wembanyama , Víctor",Ala-Pivot,CHA,20,68,68,2052,1615,672,...,79.30,108.00,858.00,966.00,194.00,58.00,102.00,177.00,147.00,88.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,NaN,"Young, Thaddeus",Ala-Pivot,CLE,35,59,29,1034,211,96,...,44.20,84.00,223.00,307.00,56.00,36.00,21.00,14.00,71.00,-200.00
521,NaN,"Young, Trae",Base,ATL,25,80,80,2400,2019,787,...,71.60,24.00,131.00,155.00,433.00,29.00,142.00,2.00,109.00,38.00
522,NaN,"Yurtseven, Omer",Pivot,BKN,N/D,64,1,1146,298,128,...,49.40,174.00,399.00,573.00,50.00,25.00,35.00,32.00,88.00,-14.00
523,NaN,"Zeller, Cody",Pivot,POR,31,38,0,328,95,43,...,45.00,59.00,124.00,183.00,23.00,5.00,7.00,3.00,20.00,-33.00


In [7]:
 ##  Scrapeamos indice de jugadores para poder sacar más info
index = aux_func.scrapping_player_index(71)
index = index.replace("PHX","PHO")

##  Cogemos los que tienen equipo actualmente
index = index[index['Equipo'] != 'Agente Libre']        
      
##  Para poder hacer merge necesitamos que los nombres esten en el mismo format
merged_stats = aux_func.invert_name_format(merged_stats)

##  Tambien hay que quitar el espacio inicial que algunos tienen
merged_stats['Jugador'] = merged_stats['Jugador'].apply(lambda x: x.strip())

##  Reemplazamos a cancar
merged_stats['Jugador'] = merged_stats['Jugador'].replace('Vlatko o Čančar','Vlatko Čančar') 

## Merge
merged_stats_final = index.merge(merged_stats, on = 'Jugador',how = 'right')

Scrappeando indice de jugadores. Espere...


100%|██████████| 71/71 [00:34<00:00,  2.04it/s]


In [8]:
##  Seleccionamos solo las columnas que vamos a usar
merged_stats_final = merged_stats_final[['Jugador','Equipo','PJ_x','MIN_x','PTS_x','FGM_x','FGA_x','3PM_x','3PA_x','TLM_x','TLA_x','REB_x','AST_x','ROB_x','PER_x','TAP_x','FP_x','R.O_x','R.D_x',
                             'PJ_y','MIN_y','PTS_y','FGM_y','FGA_y','3PM_y','3PA_y','TLM_y','TLA_y','REB_y','AST_y','ROB_y','PER_y','TAP_y','FP_y','R.O_y','R.D_y']]

In [9]:
merged_stats_final

,Jugador,Equipo,PJ_x,MIN_x,PTS_x,FGM_x,FGA_x,3PM_x,3PA_x,TLM_x,...,TLM_y,TLA_y,REB_y,AST_y,ROB_y,PER_y,TAP_y,FP_y,R.O_y,R.D_y
0,Jamal Caín,TOR,31,375,101,38,103,13,55,14,...,14.00,25.00,36.00,27.00,10.00,1.00,3.00,28.00,3.00,33.00
1,Lester Quiñones,TOR,33,556,274,93,253,51,149,37,...,30.00,42.00,39.00,65.00,6.00,16.00,3.00,23.00,4.00,35.00
2,Óscar Tshiebwe,NaN,16,233,57,25,42,0,0,7,...,7.00,13.00,106.00,12.00,1.00,4.00,6.00,9.00,36.00,70.00
3,Tristán Vukcevic,DET,12,224,91,30,84,13,52,18,...,16.00,20.00,37.00,11.00,3.00,3.00,7.00,12.00,3.00,34.00
4,Víctor Wembanyama,CHA,68,2052,1615,672,1245,64,242,207,...,195.00,246.00,966.00,194.00,58.00,102.00,177.00,147.00,108.00,858.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Thaddeus Young,CLE,59,1034,211,96,168,0,5,19,...,19.00,43.00,307.00,56.00,36.00,21.00,14.00,71.00,84.00,223.00
521,Trae Young,ATL,80,2400,2019,787,1973,233,764,206,...,197.00,275.00,155.00,433.00,29.00,142.00,2.00,109.00,24.00,131.00
522,Omer Yurtseven,BKN,64,1146,298,128,229,0,2,42,...,38.00,77.00,573.00,50.00,25.00,35.00,32.00,88.00,174.00,399.00
523,Cody Zeller,POR,38,328,95,43,61,0,0,9,...,9.00,20.00,183.00,23.00,5.00,7.00,3.00,20.00,59.00,124.00


In [10]:
##  Creamos el DF con los promedios del mes
monthly_stats = merged_stats_final
monthly_stats = monthly_stats.replace(np.nan, 0)

monthly_stats['PJ'] = monthly_stats['PJ_x'] - monthly_stats['PJ_y']
monthly_stats['MIN'] = (monthly_stats['MIN_x'] - monthly_stats['MIN_y'])/monthly_stats['PJ']
monthly_stats['PTS'] = (monthly_stats['PTS_x'] - monthly_stats['PTS_y'])/monthly_stats['PJ']
monthly_stats['REB'] = (monthly_stats['REB_x'] - monthly_stats['REB_y'])/monthly_stats['PJ']
monthly_stats['AST'] = (monthly_stats['AST_x'] - monthly_stats['AST_y'])/monthly_stats['PJ']
monthly_stats['ROB'] = (monthly_stats['ROB_x'] - monthly_stats['ROB_y'])/monthly_stats['PJ']
monthly_stats['TAP'] = (monthly_stats['TAP_x'] - monthly_stats['TAP_y'])/monthly_stats['PJ']
monthly_stats['FGM'] = (monthly_stats['FGM_x'] - monthly_stats['FGM_y'])/monthly_stats['PJ']
monthly_stats['FGA'] = (monthly_stats['FGA_x'] - monthly_stats['FGA_y'])/monthly_stats['PJ']
monthly_stats['3PM'] = (monthly_stats['3PM_x'] - monthly_stats['3PM_y'])/monthly_stats['PJ']
monthly_stats['3PA'] = (monthly_stats['3PA_x'] - monthly_stats['3PA_y'])/monthly_stats['PJ']
monthly_stats['TLA'] = (monthly_stats['TLA_x'] - monthly_stats['TLA_y'])/monthly_stats['PJ']
monthly_stats['TLM'] = (monthly_stats['TLM_x'] - monthly_stats['TLM_y'])/monthly_stats['PJ']
monthly_stats['PER'] = (monthly_stats['PER_x'] - monthly_stats['PER_y'])/monthly_stats['PJ']
monthly_stats['FP'] = (monthly_stats['FP_x'] - monthly_stats['FP_y'])/monthly_stats['PJ']
monthly_stats['ORB'] = (monthly_stats['R.O_x'] - monthly_stats['R.O_y'])/monthly_stats['PJ']
monthly_stats['DRB'] = (monthly_stats['R.D_x'] - monthly_stats['R.D_y'])/monthly_stats['PJ']
monthly_stats['FG%'] = monthly_stats['FGM']/monthly_stats['FGA']
monthly_stats['3P%'] = monthly_stats['3PM']/monthly_stats['3PA']
monthly_stats['TL%'] = monthly_stats['TLM']/monthly_stats['TLA']

monthly_stats = monthly_stats[['Jugador','Equipo','PJ','MIN','PTS','REB','AST','ROB','TAP','FGM','FGA','FG%','3PM','3PA','3P%','TLM','TLA','TL%','PER','FP','ORB','DRB']]

monthly_stats = monthly_stats.replace(np.nan, 0)


In [11]:
monthly_stats = monthly_stats[monthly_stats['PJ']>1]

In [12]:
##  Añadimos columna del TS
monthly_stats['TS%'] = monthly_stats['PTS']/(2*(monthly_stats['FGA'] + 0.44*monthly_stats['TLA']))

##  Añadimos suma pts + reb + asis para ordenar y tener una idea de los mejores
monthly_stats['Suma'] = monthly_stats['PTS'] + monthly_stats['REB'] + monthly_stats['AST']
monthly_stats = monthly_stats.sort_values(by=['Suma'],ascending=False)


In [13]:
monthly_stats.to_csv('C:/Users/luisr/OneDrive/Desktop/ANBA/23_24/Stats/stats' + fecha_ini + '-' + fecha_fin + '.csv')

In [14]:
monthly_stats.head()

,Jugador,Equipo,PJ,MIN,PTS,REB,AST,ROB,TAP,FGM,...,3P%,TLM,TLA,TL%,PER,FP,ORB,DRB,TS%,Suma
4,Víctor Wembanyama,CHA,2.00,30.00,35.00,19.00,4.00,0.50,2.50,14.00,...,0.25,6.00,7.00,0.86,0.50,1.00,4.50,14.50,0.63,58.00
463,Karl-Anthony Towns,ORL,3.00,33.00,31.00,16.00,1.00,0.33,1.67,11.33,...,0.48,4.67,5.33,0.88,1.00,4.00,1.67,14.33,0.68,48.00
264,Zach LaVine,ORL,3.00,34.00,35.67,6.33,6.00,0.00,0.00,13.33,...,0.33,5.00,5.33,0.94,0.67,1.00,1.00,5.33,0.63,48.00
512,Zion Williamson,NOP,2.00,28.50,25.50,12.00,7.50,0.00,0.50,12.00,...,0.00,1.50,4.50,0.33,0.50,2.00,3.50,8.50,0.71,45.00
107,Cade Cunningham,CLE,2.00,33.00,34.50,4.00,5.00,1.00,0.00,14.00,...,0.32,3.50,3.50,1.00,1.50,1.00,0.00,4.00,0.55,43.50


Vamos a hacer un mini Reporte

In [15]:
##  
datos_reporte = monthly_stats
datos_reporte['PJ'] = datos_reporte['PJ'].astype(int)
datos_reporte['FG%'] = datos_reporte['FG%'] * 100
datos_reporte['3P%'] = datos_reporte['3P%'] * 100
datos_reporte['GameScore'] = datos_reporte['PTS'] + 0.4 * datos_reporte['FGM'] - 0.7 * datos_reporte['FGA'] - 0.4 * (datos_reporte['TLM'] - datos_reporte['TLA']) + 0.7* datos_reporte['ORB'] + 0.3 * datos_reporte['DRB'] + datos_reporte['ROB'] + 0.7 * datos_reporte['AST'] + 0.7 * datos_reporte['TAP'] - 0.4 * datos_reporte['FP'] - datos_reporte['PER'] 

In [16]:
datos_reporte.head()

,Jugador,Equipo,PJ,MIN,PTS,REB,AST,ROB,TAP,FGM,...,TLM,TLA,TL%,PER,FP,ORB,DRB,TS%,Suma,GameScore
4,Víctor Wembanyama,CHA,2,30.00,35.00,19.00,4.00,0.50,2.50,14.00,...,6.00,7.00,0.86,0.50,1.00,4.50,14.50,0.63,58.00,35.50
463,Karl-Anthony Towns,ORL,3,33.00,31.00,16.00,1.00,0.33,1.67,11.33,...,4.67,5.33,0.88,1.00,4.00,1.67,14.33,0.68,48.00,26.63
264,Zach LaVine,ORL,3,34.00,35.67,6.33,6.00,0.00,0.00,13.33,...,5.00,5.33,0.94,0.67,1.00,1.00,5.33,0.63,48.00,28.37
512,Zion Williamson,NOP,2,28.50,25.50,12.00,7.50,0.00,0.50,12.00,...,1.50,4.50,0.33,0.50,2.00,3.50,8.50,0.71,45.00,29.60
107,Cade Cunningham,CLE,2,33.00,34.50,4.00,5.00,1.00,0.00,14.00,...,3.50,3.50,1.00,1.50,1.00,0.00,4.00,0.55,43.50,22.90


In [17]:
##  Top 10 Anotadores
datos_reporte.sort_values(by='PTS',ascending=False).head(10)[['Jugador','PJ','PTS']]

,Jugador,PJ,PTS
264,Zach LaVine,3,35.67
4,Víctor Wembanyama,2,35.00
107,Cade Cunningham,2,34.50
463,Karl-Anthony Towns,3,31.00
75,Jalen Brunson,2,30.00
144,De'Aaron Fox,6,29.50
324,Donovan Mitchell,2,29.50
330,Ja Morant,4,28.00
273,Damian Lillard,3,27.67
185,James Harden,6,27.67


In [18]:
##  Top 10 Reboteadores
datos_reporte.sort_values(by='REB',ascending=False).head(10)[['Jugador','PJ','REB']]

,Jugador,PJ,REB
126,Andre Drummond,2,21.00
83,Clint Capela,3,20.33
380,Jontay Porter,3,19.33
4,Víctor Wembanyama,2,19.00
191,Isaiah Hartenstein,2,19.00
267,Alex Len,6,16.00
463,Karl-Anthony Towns,3,16.00
510,Robert Williams,6,15.67
466,Myles Turner,3,15.67
12,Jarrett Allen,2,14.50


In [19]:
##  Top 10 Asistencias
datos_reporte.sort_values(by='AST',ascending=False).head(10)[['Jugador','PJ','AST']]

,Jugador,PJ,AST
408,Derrick Rose,4,9.50
25,Lamelo Ball,6,8.17
283,Kyle Lowry,3,7.67
512,Zion Williamson,2,7.50
155,Paul George,3,7.00
495,Coby White,2,7.00
110,Dyson Daniels,2,7.00
26,Lonzo Ball,4,6.75
147,Markelle Fultz,6,6.17
50,Anthony Black,3,6.00


In [20]:
##  Top 10 Robos
datos_reporte.sort_values(by='ROB',ascending=False).head(10)[['Jugador','PJ','ROB']]

,Jugador,PJ,ROB
433,Marcus Smart,2,3.50
442,Jaden Springer,2,2.00
110,Dyson Daniels,2,2.00
321,Justin Minaya,2,2.00
50,Anthony Black,3,2.00
395,Cam Reddish,2,2.00
325,Evan Mobley,4,2.00
10,Nickeil Alexander-Walker,3,1.67
371,Brandin Podziemski,2,1.50
428,Pascal Siakam,4,1.50


In [21]:
##  Top 10 Tapones
datos_reporte.sort_values(by='TAP',ascending=False).head(10)[['Jugador','PJ','TAP']]

,Jugador,PJ,TAP
54,Bol Bol,2,4.00
382,Kristaps Porzingis,4,3.25
4,Víctor Wembanyama,2,2.50
325,Evan Mobley,4,2.25
83,Clint Capela,3,2.00
441,Jeremy Sochan,4,1.75
226,Trayce Jackson-Davis,3,1.67
463,Karl-Anthony Towns,3,1.67
466,Myles Turner,3,1.67
429,Ben Simmons,5,1.60


In [22]:
##  Top 10 TC
datos_reporte.sort_values(by='FGM',ascending=False).head(10)[['Jugador','PJ','FGM','FGA','FG%']]

,Jugador,PJ,FGM,FGA,FG%
4,Víctor Wembanyama,2,14.00,24.50,57.14
107,Cade Cunningham,2,14.00,30.00,46.67
264,Zach LaVine,3,13.33,26.00,51.28
75,Jalen Brunson,2,12.00,23.50,51.06
512,Zion Williamson,2,12.00,16.00,75.00
144,De'Aaron Fox,6,11.50,21.00,54.76
382,Kristaps Porzingis,4,11.50,21.00,54.76
463,Karl-Anthony Towns,3,11.33,20.33,55.74
330,Ja Morant,4,11.25,20.50,54.88
273,Damian Lillard,3,11.00,22.33,49.25


In [23]:
##  Top 10 Triples
datos_reporte.sort_values(by='3PM',ascending=False).head(10)[['Jugador','PJ','3PM','3PA','3P%']]

,Jugador,PJ,3PM,3PA,3P%
193,Jordan Hawkins,2,5.00,17.00,29.41
185,James Harden,6,4.17,9.83,42.37
155,Paul George,3,4.00,10.33,38.71
79,Alec Burks,2,4.00,10.50,38.10
264,Zach LaVine,3,4.00,12.00,33.33
495,Coby White,2,4.00,8.00,50.00
446,Max Strus,2,4.00,6.00,66.67
199,Tyler Herro,4,4.00,8.00,50.00
233,Isaiah Joe,2,4.00,8.00,50.00
184,Tim Hardaway Jr.,2,4.00,9.00,44.44


In [24]:
##  Top 10 Perdidas
datos_reporte.sort_values(by='PER',ascending=False).head(10)[['Jugador','PJ','PER']]

,Jugador,PJ,PER
371,Brandin Podziemski,2,3.00
144,De'Aaron Fox,6,2.50
408,Derrick Rose,4,2.50
26,Lonzo Ball,4,2.25
25,Lamelo Ball,6,2.17
16,Cole Anthony,2,2.00
395,Cam Reddish,2,2.00
321,Justin Minaya,2,2.00
433,Marcus Smart,2,2.00
246,Tyus Jones,2,2.00


In [25]:
##  Top 10 Suma
datos_reporte.sort_values(by='Suma',ascending=False).head(11)[['Jugador','PJ','PTS','REB','AST']]

,Jugador,PJ,PTS,REB,AST
4,Víctor Wembanyama,2,35.00,19.00,4.00
264,Zach LaVine,3,35.67,6.33,6.00
463,Karl-Anthony Towns,3,31.00,16.00,1.00
512,Zion Williamson,2,25.50,12.00,7.50
107,Cade Cunningham,2,34.50,4.00,5.00
382,Kristaps Porzingis,4,26.00,14.00,2.00
428,Pascal Siakam,4,24.25,13.00,2.25
144,De'Aaron Fox,6,29.50,3.83,5.83
324,Donovan Mitchell,2,29.50,3.50,5.50
273,Damian Lillard,3,27.67,4.67,5.00


In [26]:
##  Top 10 Game Score
datos_reporte.sort_values(by='GameScore',ascending=False).head(10)[['Jugador','PJ','GameScore']]

,Jugador,PJ,GameScore
4,Víctor Wembanyama,2,35.50
512,Zion Williamson,2,29.60
264,Zach LaVine,3,28.37
463,Karl-Anthony Towns,3,26.63
382,Kristaps Porzingis,4,23.87
144,De'Aaron Fox,6,23.37
107,Cade Cunningham,2,22.90
185,James Harden,6,21.48
182,Tyrese Haliburton,4,20.90
75,Jalen Brunson,2,20.90


In [27]:
## Wizards
datos_wizards = datos_reporte[datos_reporte['Equipo'] == 'WAS']

In [28]:
datos_wizards.sort_values(by='Suma',ascending=False).head(5)[['Jugador','PJ','MIN','PTS','REB','AST']]

,Jugador,PJ,MIN,PTS,REB,AST
379,Craig Porter,2,36.00,22.00,5.50,5.00
371,Brandin Podziemski,2,46.00,20.00,6.00,2.50
524,Ivica Zubac,2,19.50,9.50,10.50,1.00
502,Alondes Williams,2,30.00,9.00,4.50,1.50
263,Jake LaRavia,2,30.50,7.50,5.00,2.00


: 

In [ ]:
##  Idem com Grizzlies
datos_grizlies = datos_reporte[datos_reporte['Equipo'] == 'MIN']
datos_grizlies.sort_values(by='Suma',ascending=False).head(5)[['Jugador','PJ','MIN','PTS','REB','AST']]

Vamos a hacer Reportes para stats por 36 min

In [ ]:
##  Scrapeamos stats jugadores
season = '2023-2024'
stats_players = aux_func.scrapping_stats_players(season, 27)

In [ ]:
stats_players2 =stats_players

In [ ]:
##  Sacamos min por partido
stats_players2['MPP'] = stats_players2['MIN']/stats_players2['PJ'] 

In [ ]:
##  Quitamos los que hayan jugado menos de 10 partidos y menos de 10 min por partido
stats_players2 = stats_players2[stats_players2['PJ']>10]
stats_players2 = stats_players2[stats_players2['MPP']>10]

In [ ]:
stats_players2.head()

In [ ]:
##Calculamos valores por 36 min

## PUNTOS
stats_players2['a'] = (stats_players2['3PA']/stats_players2['MIN'])*36
stats_players2.sort_values(by='a',ascending=False).head(10)[['Jugador','a']]